# Project

## Summary of the course
In this course we will explore momentum-based trading strategies using the geometric Brownian motion as the model. I will explain why a quantitative, rigorous model is useful and how it helps us forecast asset prices in the future, how such a forecast can be used to make trading decisions.

I will also explain the famous Black-Schole's formula for European option pricing, which is also based on the geometric Brownian motion and which helps us hedge risks wherever needed. After explaining the math, I will also show you how to use Python, and the numpy and scipy packages in particular, to program the trading strategy. You can then expand and customise the strategy to suit your needs. In addition, I will introduce you to the concepts and techniques of quantitative risk management, including the calculation of the *Value at Risk* and the *Expected Shortfall*. Their application will add an extra layer of safety net to the trading strategy.

Tips: A review of calculus including differentiation and integration will give you a head start in the course!



Re-organize your code from the exercises into a Python program that 
1. read prices from a database
2. calibrate a GBM model on each day when new prices are received.
3. forecast the price in e.g. 10 days and
   1. calculate the confidence interval of the forecast
   2. calculate the expected shortfall from the price forecast
4. code your trading signals using the price forecast and the expected shortfall.
5. store your postions into the *positions* table after each trade.
6. produce a 1-year backtest record from 2020-06-01 to 2021-05-31.

**Hint**

1. Collect all the code related to the GBM model into a class

In [1]:
import numpy as np
import csv
import sqlite3
from scipy.stats import norm
from contextlib import closing

conn = None
cs = None

class GBM:
    def __init__(self):
        self.mu = np.nan;
        self.sigma = np.nan;
        self.rng = np.random.default_rng()
        
    def simulate(self, N, K, Dt, S0):
        sqrt_Dt = np.sqrt(Dt)
        traj = np.full((N+1, K), np.nan)
        drift = (self.mu - self.sigma**2/2) * np.linspace(1, N, N) * Dt
        for i in range(K):
            W = sqrt_Dt * np.cumsum(norm.rvs(size=N))
            traj[1:, i] = S0 * np.exp(drift + self.sigma * W)
            traj[0, i] = S0
        return traj;

    def calibrate(self, trajectory, Dt):
        increments = np.diff(np.log(trajectory));
        moments = [0, 0];
        n_iter = 10;
        for iter in range(n_iter):
            X = self.rng.choice(increments, size=len(increments)//2)
            moments[0] += np.mean(X)/n_iter;
            moments[1] += np.mean(X**2)/n_iter
        std = np.sqrt(moments[1] - moments[0]**2);
        self.sigma = std/np.sqrt(Dt);
        self.mu = moments[0] / Dt + self.sigma**2/2;

    def forecast(self, latest, T, confidence):
        m = (self.mu - self.sigma**2/2)/2 * T;
        s = self.sigma * np.sqrt(T);
        Q = norm.ppf([(1-confidence)/2, (1+confidence)/2], loc=m, scale=s)
        return {
            'confidence': confidence,
            'expected': latest * np.exp(self.mu * T),
            'interval': latest * np.exp(Q)
        };

    def expected_shortfall(self, T, confidence):
        m = (self.mu - self.sigma**2/2)/2 * T;
        s = self.sigma * np.sqrt(T);
        ES = -m + s * norm.pdf(norm.ppf(confidence))/(1 - confidence);
        return ES;



2. Write a function that prepares the database for trading, i.e.
   1. load the historical prices into the *prices* table
   2. create the *positions* table
   3. initialize the *positions* table with the your initial cash reserve. The initial *time_of_trade* can be any date before the earliest possible trading date.

    Call this function *prepare*.

In [2]:
def prepare():
    cs.execute("""
    create table if not exists prices (
    theday text primary key,
    price real
    );
    """)
    with closing(open('SP500.csv')) as datafile:
        reader = csv.DictReader(datafile, fieldnames=["date", "price"], delimiter='\t')
        for row in reader:
            cs.execute(F"""
            insert or ignore into prices values (\"{row['date']}\",
            {float(row['price'])});
            """)
    cs.execute(F"""
    create table if not exists positions (
    time_of_trade text,
    instrument text,
    quantity real,
    cash real,
    primary key (time_of_trade, instrument)
    );
    """);
    cs.execute(F"""
    insert or ignore into positions values
    ('1666-01-01', 'SP500', 0, 1000000);
    """)
    conn.commit()


3. Write a function that determines the trade size, i.e. how many units of the instrument you would like to own when the date is *which_day* and the price forecast of the instrument is *forecast* and the expected shortfall from the same forecast is *ES*.

In [8]:
def position_size(which_day, forecast):
    cs.execute(F"""
    select quantity, cash from positions
    where instrument = 'SP500'
    and time_of_trade < '{which_day}'
    order by time_of_trade desc
    limit 1;
    """)
    qty, cash = cs.fetchall()[0]
    cs.execute(F"""
    select price from prices
    where theday <= '{which_day}'
    order by theday desc
    limit 1;
    """);
    price = cs.fetchall()[0][0]
    if price < forecast['interval'][0]:
        return qty + round(cash/price)
    elif price > forecast['interval'][1]:
        return -qty
    else:
        return qty


4. Write a function that, for a given date, calibrates a GBM model to the data prior to that date and that forecasts the price in 10 days. Call this function *analyse*.

In [9]:
def analyse(which_day):
    cs.execute(F"""
    select price from prices where theday <= '{which_day}'
    order by theday desc limit 120;
    """)
    P = np.flipud(np.asarray(cs.fetchall())).flatten();
    model = GBM();
    Dt = 1.0/252;
    model.calibrate(P, Dt);
    confidence = 0.1
    T = 10 * Dt;
    forecast = model.forecast(P[-1], T, confidence);
    return position_size(which_day, forecast);
    


5. The main loop of the program: Loop over the dates in the backtest period and use the *analyse* function to decide what to do on each day. Call this function *main*.

In [10]:
def main(begin_on):
    cs.execute(F"select theday from prices where theday >= '{begin_on}';")
    days = [d[0] for d in cs.fetchall()]
    asset = {
        'old': np.nan,
        'new': np.nan
    };
    cash = {
        'old': np.nan,
        'new': np.nan
    };
    cs.execute("delete from positions where time_of_trade > '2020-01-01';");
    for d in days:
        asset['new'] = analyse(d)
        cs.execute(F"""
        select quantity, cash from positions
        where time_of_trade < '{d}'
        order by time_of_trade desc
        limit 1;
        """);
        asset['old'], cash['old'] = cs.fetchall()[0];
        cs.execute(F"""
        select price from prices
        where theday <= '{d}'
        order by theday desc
        limit 1;
        """);
        latest = cs.fetchall()[0][0]
        trade_size = round(asset['new']) - round(asset['old']);
        if trade_size != 0:
            cash['new'] = cash['old'] - trade_size * latest;
            cs.execute(F"""
            insert into positions values
            ('{d}', 'SP500', {round(asset['new'])}, {cash['new']});
            """);
        conn.commit();



6. Connect to the database and create a *cursor* object associated with the connection. Share the connection and the cursor object across the program so that you don't have to connect to and disconnect from the database in every function of the program.

In [11]:
if __name__ == "__main__":
    with closing(sqlite3.connect("SP500.db")) as conn:
        with closing(conn.cursor()) as cs:
            prepare()
            main('2020-06-01')
    
